In [1]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader

# 하이퍼파라미터 설정
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# 데이터셋 전처리
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),    # 랜덤으로 이미지를 좌우 반전
    transforms.RandomCrop(32, padding=4), # 랜덤 크롭 및 패딩
    transforms.ToTensor(),                # 텐서로 변환
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # 정규화
])

# CIFAR-10 데이터셋 로드
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 정의 (ResNet18 사용)
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)  # CIFAR-10 클래스 수에 맞게 마지막 레이어 조정

# GPU 사용 가능하면 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파 및 옵티마이저 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(train_loader)

# 테스트 함수
def test(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # 예측 및 손실 계산
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 모델 학습 및 테스트
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    test_accuracy = test(model, test_loader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

print("모델 학습 완료!")


Files already downloaded and verified
Files already downloaded and verified


c:\ProgramData\Anaconda3\envs\mjjenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\mjjenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/10], Loss: 1.5549, Test Accuracy: 0.5329
Epoch [2/10], Loss: 1.1902, Test Accuracy: 0.5715
Epoch [3/10], Loss: 1.0298, Test Accuracy: 0.6191
Epoch [4/10], Loss: 0.9302, Test Accuracy: 0.6925
Epoch [5/10], Loss: 0.8479, Test Accuracy: 0.7058
Epoch [6/10], Loss: 0.7883, Test Accuracy: 0.7227
Epoch [7/10], Loss: 0.7423, Test Accuracy: 0.7253
Epoch [8/10], Loss: 0.7044, Test Accuracy: 0.7358
Epoch [9/10], Loss: 0.6760, Test Accuracy: 0.7534
Epoch [10/10], Loss: 0.6397, Test Accuracy: 0.7628
모델 학습 완료!
